In [1]:
##CREATE A VARIABLE INPUT for the DNN save the W and b and feed again // change Batch size

import requests
import urllib.request
import urllib.parse
import json
from bs4 import BeautifulSoup
from pymongo import MongoClient
import re
import os

#DB connexion
global client 
client = MongoClient(port=27017)
global db
db=client.GeniusScrappingProject

#genius API base and client_token
base = "https://api.genius.com"
client_access_token = "z1va0QLLfH2tm_2NErEGiCUvLUJS0RMBMzfx17EEzUnj3wvJOu4_Dvjx6CNJyJSW"

In [21]:
def save_lyrics_txt():
    song_ids = get_existing_songs()
    for song_id in song_ids:
        lyrics = retrieve_lyrics(song_id[0])
        with open(os.path.expanduser("Lyrics.txt"),"ab") as f:
            f.write(bytes(lyrics,encoding="ascii", errors='ignore'))
 

In [22]:
save_lyrics_txt()

In [11]:
#DB Get requests#
def get_existing_songs():
	#Returns a list of the songs and their primary artists#
	songs = db.songs.find()
	existing_songs = []
	for song in songs:
		existing_songs.append([song['id'], song['primary_artists'].items()])
	return existing_songs

def get_songs_with_lyrics_scrapped():
	#Returns the list of songs ids that lyrics has been already added to DB#
	songs_scrapped =[]
	lyrics = db.lyrics.find()
	for song_id in lyrics:
		songs_scrapped.append(song_id['song_id'])
	return songs_scrapped

def get_primary_artists_from_songs():
	#Returns the primary artist of all the songs in the DB#
	songs = db.songs.find()
	existing_songs = {}
	for i, song in enumerate(songs):
		existing_songs.update({
		song['primmary_artists']
		})
	return existing_songs

#returns list of IDs of the artists we have in the DB
def get_list_artists():
#Returns all artists from DB#
	artists = db.artists.find()
	existing_artists = []
	for artist in artists:
		existing_artists.append(artist['id'])
	return existing_artists

def get_artists_without_songs_in_db():
	#Returns artists that don't have songs in ou DB#
	all_artists = get_list_artists()
	artists_with_songs = get_artists_with_songs_in_db()
	artists_without_songs = []
	for artist in all_artists:
		if not artist in artists_with_songs:
			artists_without_songs.append(artist)
	return artists_without_songs

def get_artists_with_songs_in_db():
	#Returns artists that have songs in ou DB#
	existing_songs = get_existing_songs()
	artists_with_songs=[]
	list_atists = get_list_artists()
	for song in existing_songs:
		if sorted(song[1])[0][1] in list_atists:
			artists_with_songs.append(sorted(song[1])[0][1])
	artists_with_songs = list(set(artists_with_songs))
	return artists_with_songs

def get_verses_from_lyrics(lyrics):
	#Returns a list of verses from lyrics#
	verses = []
	lyrics_keys = list(lyrics.keys())
	for key in lyrics_keys:
		if is_tagged_verse(key):
			verses.append(lyrics[key])
	return verses

def get_all_lyrics():
	#Returns the lyrics from DB#
	existing_lyrics = db.lyrics.find()
	all_lyrics = []
	for lyrics in existing_lyrics:
		all_lyrics.append(lyrics)
	return all_lyrics


In [12]:
'''DB Insert requests'''
#add new artists to our DB
def add_artists(artists):
    for artist in artists:
        entry = {
            'id' : int(artists[artist]['id']),
            'name' : artists[artist]['name'],
            'url' : artists[artist]['url'],
        }
        #Step 3: Insert business object directly into MongoDB via isnert_one
        result=db.artists.insert_one(entry)
        print('artists {} added with success'.format(artists[artist]['name']))

def add_songs(songs):
    '''add new songs to the DB'''
    existing_songs = get_existing_songs()
    entry={}
    for i in songs:
        if not songs[i]['id'] in existing_songs: 
            entry = {
                "id" : songs[i]['id'],
                "url": songs[i]["url"],
                "title": songs[i]["title"],
                "primary_artists" : songs[i]["primary_artists"],
                "album": songs[i]["album"],
                "release_date": songs[i]["release_date"],
                "featured_artists":
                    {feat : songs[i]["featured_artists_id"][j] 
                     if songs[i]["featured_artists"] else "" for j, feat in enumerate(songs[i]["featured_artists"])},
                #"featured_artists_id":
                #    [feat if songs[i]["featured_artists"] else "" for feat in songs[i]["featured_artists_id"]],
                "genius_track_id": songs[i]["genius_track_id"],
                "genius_album_id": songs[i]["genius_album_id"]
            }
            #Step 3: Insert business object directly into MongoDB via isnert_one
            result=db.songs.insert_one(entry)
            #Step 4: Print to the console the ObjectID of the new document
            print('added to DB as {0}'.format(songs[i]['title']))
    #Step 5: Tell us that you are done
    print('finished adding songs')

def add_lyrics():
    '''add lyrics of the songs in DB'''
    songs = get_existing_songs()
    song_ids=[]
    songs_not_to_scrap = get_songs_with_lyrics_scrapped()
    for i,song in enumerate(songs):
        if not song[0] in songs_not_to_scrap:
            parts_indexes = []
            parts_labels = []
            dic_lyrics = {}
            song_parts =[]

            #Get the lyrics of the song
            full_lyrics = retrieve_lyrics(song[0])

            #we can manipulate the lyrics and devide them into bars
            splitted_lyrics=re.split("\n+", full_lyrics)
            for j,lyric in enumerate(splitted_lyrics):
                if re.search('\[(.*)\]',lyric):
                    parts_indexes.append(j)
                    lyric = re.search('\[(.*)\]',lyric)
                    parts_labels.append(lyric.group(1))
                    part_of_the_song = ''
                    for k,search_for_next_part in enumerate(splitted_lyrics[j+1:]):
                        next_part_found = False
                        if not next_part_found:
                            if re.search('\[(.*)\]',search_for_next_part) or k>len(splitted_lyrics)-5:
                                next_part_found = True
                                j=k-1
                                break
                            else:
                                '''concatinating lyrics'''
                                part_of_the_song = part_of_the_song+'\n'+search_for_next_part
                    song_parts.append(part_of_the_song)
        
        dic_lyrics['song_id'] = int(song[0])
        for i, indexes in enumerate(parts_indexes):
            dic_lyrics[re.sub(r"[^\w\s]", ' ',parts_labels[i])] = song_parts[i]
            if is_tagged_verse(r"[^\w\s]", ' ',parts_labels[i]):
                result = db.verses.insert_one({'song_id':int(song[0]),'verse':song_parts[i]})
        if parts_indexes:
            result=db.lyrics.insert_one(dic_lyrics)
            print(dic_lyrics)
        

def add_verses():
    '''Add the verses to The DB with the song ID'''
    lyrics = get_all_lyrics()
    for lyric in lyrics:
        print(lyric['song_id'])
        verses=[]
        verses=get_verses_from_lyrics(lyric)
        for verse in verses:
            db.verses.insert_one({'song_id' : lyric['song_id'],'verse' : verse})

In [3]:
        #Genius API requests
def get_json(path, params=None, headers=None):
	# Generate request URL
	requrl = '/'.join([base, path])
	token = "Bearer {}".format(client_access_token)
	if headers:
		headers['Authorization'] = token
	else:
		headers = {"Authorization": token}

	# Get response object from querying genius api
	response = requests.get(url=requrl, params=params, headers=headers)
	response.raise_for_status()
	return response.json()

def search(artist_name):
	#Search Genius API via artist name.#
	search = "/search?q="
	query = base + search + urllib.parse.quote(artist_name)
	print(query)
	request = urllib.request.Request(query)

	request.add_header("Authorization", "Bearer " + client_access_token)
	request.add_header("User-Agent", "")
	
	response = urllib.request.urlopen(request, timeout=3)
	raw = response.read()
	data = json.loads(raw)['response']['hits']

	for item in data:
		# Print the artist and title of each result
		print(item['result']['primary_artist']['name']
			  + ': ' + item['result']['title'])

def search_artist(artist_id):
	#Search meta data about artist Genius API via Artist ID.#
	search = "artists/"
	path = search + str(artist_id)
	request = get_json(path)
	data = request['response']['artist']

	print(data["followers_count"])
	# Lots of information we can scrape regarding the artist, check keys
	return data["followers_count"] # number of followers

def connect_lyrics(song_id):
	#Constructs the path of song lyrics.#
	url = "songs/{}".format(song_id)
	data = get_json(url)

	# Gets the path of song lyrics
	path = data['response']['song']['path']

	return path

def retrieve_lyrics(song_id):
	#Retrieves lyrics from html page.#
	path = connect_lyrics(song_id)

	URL = "http://genius.com" + path
	page = requests.get(URL)

	# Extract the page's HTML as a string
	html = BeautifulSoup(page.text, "html.parser")

	# Scrape the song lyrics from the HTML
	lyrics = html.find("div", class_="lyrics").get_text()
	return lyrics



def get_song_id(artist_id):
	#Get all the song id from an artist.#
	current_page = 1
	next_page = True
	songs = [] # to store final song ids

	while next_page:
		path = "artists/{}/songs/".format(artist_id)
		params = {'page': current_page} # the current page
		data = get_json(path=path, params=params) # get json of songs
		page_songs = data['response']['songs']
		if page_songs:
			# Add all the songs of current page
			songs += page_songs
			# Increment current_page value for next loop
			current_page += 1
			print("Page {} finished scraping".format(current_page))
			# If you don't wanna wait too long to scrape, un-comment this
			#if current_page == 2:
			 #   break
		else:
			# If page_songs is empty, quit
			next_page = False

	print("Song id were scraped from {} pages".format(current_page))

	# Get all the song ids, excluding not-primary-artist songs.
	songs = [song["id"] for song in songs
	if song["primary_artist"]["id"] == artist_id]

	return songs

#get the informations of a song
def get_song_information(song_ids):
	song_list = {}
	print("Scraping song information")
	for i, song_id in enumerate(song_ids):
		print("id:" + str(song_id) + " start. ->")

		path = "songs/{}".format(song_id)
		data = get_json(path=path)["response"]["song"]

		song_list.update({
		i: {
			"id" : data['id'],
			"url": data["url"],
			"title": data["title"],
			"primary_artists" : {re.sub(r"[^\w\s]", ' ', data["primary_artist"]['name']):data["primary_artist"]['id']},
			"album": data["album"]["name"] if data["album"] else "<single>",
			"release_date": data["release_date"] if data["release_date"] else "unidentified",
			"featured_artists":
			[re.sub(r"[^\w\s]", ' ', feat["name"]) if data["featured_artists"] else "" for feat in data["featured_artists"]],
			"featured_artists_id":
			[feat['id'] if data["featured_artists"] else "" for feat in data["featured_artists"]],
			"genius_track_id": song_id,
			"genius_album_id": data["album"]["id"] if data["album"] else "none"}
		})

		print("-> id:" + str(data['title']) + " is finished. \n")
	return song_list

#def get_producers():


def get_other_artists_from_songs(song_ids):
	#Retrieve meta data about a song.#
	# initialize existing artists and a dictionary for new artists.
	existing_artists = get_list_artists()
	artist_list ={}
	print("Scraping song information")
	for i, song_id in enumerate(song_ids):
		print("id:" + str(song_id) + " start. ->")
		path = "songs/{}".format(song_id)
		data = get_json(path=path)["response"]["song"]
		if data["featured_artists"]:
			for artist in data["featured_artists"]:
				if not artist['id'] in existing_artists:
					artist_list.update({
					int(artist["id"]): {
					"id": int(artist["id"]),
					"url": artist["url"],
					"name": artist["name"]}
					})
		if data["primary_artist"]:
			if not data["primary_artist"]["id"] in existing_artists:
				artist_list.update({
				int(data["primary_artist"]["id"]): {
				"id": int(data["primary_artist"]["id"]),
				"url": data["primary_artist"]["url"],
				"name": data["primary_artist"]["name"]}
				})

		print("-> id:" + str(song_id) + " is finished. \n")

	return artist_list  

def get_songs_of_all_artists(artists_ids, artist_without_songs, existing_songs):
	#Grabs all song id's from artists# 
	for artist_id in artists_ids:
		if artist_id in artist_without_songs:
			songs_ids=[]
			song_list = []
			songs_ids_of_an_artist = get_song_id(artist_id)
			for song in songs_ids_of_an_artist:
	#To not scrap songs we already have#
				if not song in existing_songs:
					songs_ids.append(song)
					song_list = get_song_information(songs_ids)
					add_songs(song_list)
					print('songs added to DB')
	print('done')


In [23]:
def is_tagged_verse(word):
    if re.search('(V|v)erse',str(word)):
        return True
    else:
        return False

def is_tagged_chorus(word):
    if re.search('(C|c)horus',str(word)):
        return True
    else:
        return False

In [4]:
search_artist(45)

24027


24027

In [24]:
def main():
    # Example searches
    term = 'Eminem'
    artist_list = {}
    artist_without_songs = get_artists_without_songs_in_db()
    artists_with_songs = get_artists_with_songs_in_db()
    artists_ids = get_list_artists()
    existing_songs = get_existing_songs()

            #Grabs all song id's from one artist
    songs_ids=get_song_id(56)
    
            # Get meta information about songs
    #song_list = get_song_information(songs_ids)
    
            # Add songs of the artist to the DB
    #add_songs(song_list)
    
            #Grab all featuring artists from songs ID
    #artist_list = get_other_artists_from_songs(songs_ids)
    
            #add new artists to the DB
    #add_artists(artist_list)
    
            # Grabs all song id's from artists
    #for artist_id in artists_ids:
#        if artist_id in artist_without_songs:
#            songs_ids=[]
#            song_list = []
#            songs_ids_of_an_artist = get_song_id(artist_id)
#            for song in songs_ids_of_an_artist:
#                '''To not scrap songs we already have'''
#                if not song in existing_songs:
#                    songs_ids.append(song)
#            song_list = get_song_information(songs_ids)
#            add_songs(song_list)
    
            # Scrape lyrics from the songs
    # song_lyrics = [retrieve_lyrics(song_id) for song_id in songs_ids]


            # Gets information regarding the artist themself
    # followers = search_artist(artist_id)

            # Shows some random songs from arist and lyrics
    #search(term)


if __name__ == "__main__":
    main()

Page 2 finished scraping
Page 3 finished scraping
Page 4 finished scraping
Page 5 finished scraping
Page 6 finished scraping
Page 7 finished scraping
Page 8 finished scraping
Page 9 finished scraping
Page 10 finished scraping
Page 11 finished scraping
Page 12 finished scraping
Page 13 finished scraping
Page 14 finished scraping
Page 15 finished scraping
Page 16 finished scraping
Page 17 finished scraping
Page 18 finished scraping
Page 19 finished scraping
Page 20 finished scraping
Page 21 finished scraping
Page 22 finished scraping
Page 23 finished scraping
Page 24 finished scraping
Page 25 finished scraping
Page 26 finished scraping
Page 27 finished scraping
Page 28 finished scraping
Page 29 finished scraping
Page 30 finished scraping
Page 31 finished scraping
Page 32 finished scraping
Page 33 finished scraping
Page 34 finished scraping
Page 35 finished scraping
Page 36 finished scraping
Page 37 finished scraping
Page 38 finished scraping
Page 39 finished scraping
Page 40 finished scr

-> id:Hip Hop Is Dead Single Art is finished. 

id:34794 start. ->
-> id:Hold Down the Block is finished. 

id:3665 start. ->
-> id:Hope is finished. 

id:56243 start. ->
-> id:Hustlers is finished. 

id:105900 start. ->
-> id:I Already Know is finished. 

id:246825 start. ->
-> id:I Am... Album Art is finished. 

id:3167197 start. ->
-> id:I Am... [Credits] is finished. 

id:439485 start. ->
-> id:I Am... (Intro) is finished. 

id:614 start. ->
-> id:I Can is finished. 

id:705367 start. ->
-> id:I Can (Remix) is finished. 

id:191036 start. ->
-> id:If I Ruled The World ’09 is finished. 

id:1590 start. ->
-> id:If I Ruled the World (Imagine That) is finished. 

id:465651 start. ->
-> id:If I Ruled the World (Imagine That) Single Art is finished. 

id:953 start. ->
-> id:I Gave You Power is finished. 

id:355907 start. ->
-> id:Illmatic 10th Anniversary Platinum Edition Album Art is finished. 

id:196333 start. ->
-> id:Illmatic [Credits] is finished. 

id:246265 start. ->
-> id:Illm

-> id:One Mic (Remix) is finished. 

id:34853 start. ->
-> id:One Never Knows is finished. 

id:34856 start. ->
-> id:One on One is finished. 

id:731 start. ->
-> id:One Time 4 Your Mind is finished. 

id:3667839 start. ->
-> id:One Time 4 Your Mind (Live) is finished. 

id:3461 start. ->
-> id:On the Real is finished. 

id:53263 start. ->
-> id:On The Real (original) is finished. 

id:3085050 start. ->
-> id:On The Road Again is finished. 

id:34870 start. ->
-> id:Oochie Wally is finished. 

id:104966 start. ->
-> id:Open Letter to Young Warriors in Chicago (Sep. 2009) is finished. 

id:379695 start. ->
-> id:Peace God is finished. 

id:51699 start. ->
-> id:Play on Playa is finished. 

id:1814247 start. ->
-> id:Play On Playa [Clean] is finished. 

id:34817 start. ->
-> id:Poppa Was a Playa is finished. 

id:2745 start. ->
-> id:Power Paper Pussy is finished. 

id:34804 start. ->
-> id:Pray is finished. 

id:34887 start. ->
-> id:Proclamation (Nigger Hatred) is finished. 

id:34890

-> id:Untitled is finished. 

id:34868 start. ->
-> id:U Wanna Be Me is finished. 

id:2110297 start. ->
-> id:U wanna be me - soundtrack version is finished. 

id:4672342 start. ->
-> id:Vernon Family is finished. 

id:29540 start. ->
-> id:Virgo is finished. 

id:1819744 start. ->
-> id:Virgo (alternate version) is finished. 

id:1266805 start. ->
-> id:Wanna Be Me is finished. 

id:1448159 start. ->
-> id:Wanna Play is finished. 

id:34801 start. ->
-> id:Wanna Play (Rough?) is finished. 

id:34772 start. ->
-> id:War is finished. 

id:4672351 start. ->
-> id:War Against Love is finished. 

id:2869871 start. ->
-> id:War (Birth Of A Nation) is finished. 

id:34869 start. ->
-> id:War Is Necessary is finished. 

id:34787 start. ->
-> id:Warrior Song is finished. 

id:2913 start. ->
-> id:Watch Dem Niggas is finished. 

id:34871 start. ->
-> id:We March as Millions is finished. 

id:3944 start. ->
-> id:We’re Not Alone is finished. 

id:3472 start. ->
-> id:We Will Survive is finished

added to DB as Poppa Was a Playa
added to DB as Power Paper Pussy
added to DB as Pray
added to DB as Proclamation (Nigger Hatred)
added to DB as Project Roach
added to DB as Project Windows
added to DB as Purple
added to DB as Pussy Killz (Bonus Disc)
added to DB as QueensBridge Politics
added to DB as Queens Get the Money
added to DB as Queens Wolf
added to DB as Quiet Niggas
added to DB as Rare/Unreleased Tracks & Freestyles From Pre-Illmatic To The Present
added to DB as Reach Out
added to DB as Real Niggas
added to DB as Reason
added to DB as Remember the Times
added to DB as Remember the Times (Intro)
added to DB as Remember The Times (Intro) - Explicit Album Version
added to DB as Represent
added to DB as Represent (Live)
added to DB as Rest of my Life
added to DB as Revolutionary Warfare
added to DB as Rewind
added to DB as Rise & Fall
added to DB as Roses
added to DB as Royalty
added to DB as Rule
added to DB as Salute Me
added to DB as Salute Me (The General)
added to DB as Se

In [ ]:
#Eminem 553 songs Nas 445 songs 

3289822
1864019
202554
695536
4590186
2463747
5024
563
391
603
18903
129149
5760
914
58429
49065
3207971
3103228
203924
54563
50059
203925
268736
3103229
70048
4755292
3407396
2414929
537
2129
93360
3913239
422
739388
72764
3350704
235728
958
1086
239829
18885
235723
3350690
1495
513
413066
1031
4914
545
239834
4020873
3350675
459967
210094
57098
1998
1927
214804
4433932
1203
739384
235730
739391
808
868
4755072
2917
921
4755105
3697403
1725
2331
3350703
4494798
3350679
3420343
570
873
3413354
4755155
3633583
2843
533
3932765
1314
690601
4531773
1627
1304
554
1792063
617
156489
2430673
1790620
3417
239831
479
576535
4889502
2343861
586212
3002
409402
218092
1969
3889084
474360
252542
1601
2297
690
3775
1857
255983
140754
3930603
943114
96835
18924
941
591453
2014
1999
215523
209365
1489
235736
2053
3930586
393426
1790615
2479944
156337
586227
141803
4548
4753567
3350692
4061099
2403657
80365
232922
367772
2945258
363777
78099
4759431
75173
2148737
4933
1938
2784
3969
4981
2982910
574
3